In [1]:
## Components

from collections import namedtuple


In [2]:
# config entity
DataIngestionConfig = namedtuple("DataIngestionConfig",[
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [2]:
### Alternatives to name class

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)   ## frozen = True makes it behave like named tuple,we won't be able to change anything
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir:Path
    

In [3]:
from deepClassifier.constants import *
from deepClassifier.utils import *

In [4]:
import os

In [5]:
os.chdir('..')

# Creating Configuration Manager Class

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH,
                 ) -> None:
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [7]:
configuration_manager = ConfigurationManager()

[2022-09-17 10:34:43,300: INFO : common]: yaml file: configs\config.yaml loaded successfully
[2022-09-17 10:34:43,305: INFO : common]: yaml file: params.yaml loaded successfully
[2022-09-17 10:34:43,307: INFO : common]: created directory at: artifacts


In [8]:
configuration_manager.get_data_ingestion_config()

[2022-09-17 10:34:43,806: INFO : common]: created directory at: artifacts/data_ingestion


DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')

# Data Ingestion Class

In [9]:
import urllib.request as request
from zipfile import ZipFile

In [10]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig) -> None:
        self.config = config
        
    def download_file(self):
        # Will only download in case the file isn't downloaded yet
        
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]
            
    
    def _preprocess(self,zf:ZipFile, f:str, working_dir:str):
        target_filepath = os.path.join(working_dir,f)
        
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)
    
    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode='r') as zf:
            list_of_files = zf.namelist()
            update_list_of_files = self._get_updated_list_of_files(list_of_files=list_of_files)
            
            for f in update_list_of_files:
                self._preprocess(zf=zf, f = f, working_dir=self.config.unzip_dir)
            


In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2022-09-17 10:34:45,604: INFO : common]: yaml file: configs\config.yaml loaded successfully
[2022-09-17 10:34:45,605: INFO : common]: yaml file: params.yaml loaded successfully
[2022-09-17 10:34:45,607: INFO : common]: created directory at: artifacts
[2022-09-17 10:34:45,609: INFO : common]: created directory at: artifacts/data_ingestion
